In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/stockdata/new.csv
/kaggle/input/stockdata/top.csv
/kaggle/input/stockdata/all.csv
/kaggle/input/stockdata/Data/modIndex.csv
/kaggle/input/stockdata/Data/Index.csv
/kaggle/input/stockdata/Data/RiskFreeRate.csv
/kaggle/input/stockdata/Data/Equity.csv
/kaggle/input/stockdata/Data/Stock/fc500770.csv
/kaggle/input/stockdata/Data/Stock/532540.csv
/kaggle/input/stockdata/Data/Stock/fc500325.csv
/kaggle/input/stockdata/Data/Stock/fc532540.csv
/kaggle/input/stockdata/Data/Stock/533155.csv
/kaggle/input/stockdata/Data/Stock/500180.csv
/kaggle/input/stockdata/Data/Stock/fc500085.csv
/kaggle/input/stockdata/Data/Stock/fc533155.csv
/kaggle/input/stockdata/Data/Stock/fc533260.csv
/kaggle/input/stockdata/Data/Stock/gr500124.csv
/kaggle/input/stockdata/Data/Stock/gr500182.csv
/kaggle/input/stockdata/Data/Stock/gr500209.csv
/kaggle/input/stockdata/Data/Stock/fc532210.csv
/kaggle/input/stockdata/Data/Stock/532538.csv
/kaggle/input/stockdata/Data/Stock/fc500680.csv
/kaggle/input/stockdata/D

In [2]:
import numpy as np
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from math import sqrt
from sklearn.neighbors import DistanceMetric
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import precision_score, make_scorer,f1_score
import traceback
import prettytable


In [3]:
import warnings; warnings.simplefilter('ignore')

In [4]:
def pre_process_data(data,null_threshold):
    """
    Drops Date and Unix Date columns from the data.
    Drops the columns which has null values more than specified null_threshold.
    Replaces infinite values with NAN.
    Drops the rows which has null values.

    Parameters
    ----------
    data : dataframe

    null_threshold : numeric
        numeric value describing the amount of null values that can be present.

    Returns
    -------
    data : dataframe
        an updated dataframe after performing all the opertaions.
    """
    
    data.drop(columns=['Unix Date','Date'],axis=1,inplace=True)
    total = data.shape[0]
    for col in data.columns:
        if null_threshold * total / 100 < data[col].isnull().sum():
            data.drop(columns=[col],axis=1,inplace=True)
    data.replace([np.inf, -np.inf], np.nan, inplace=True)
    data = data.apply(pd.to_numeric,errors='coerce')
    data.dropna(axis=0,inplace=True)
    return data

In [5]:
def dependent_column(data,column):
    """
    Removes all the Next Day columns.
    Removes all the non Growth Rate Columns (GR)
    add the predictor column to list of columns.

    Parameters
    ----------
    data : dataframe

    column : string
        name of the predictor column 

    Returns
    -------
    data : dataframe
        an updated dataframe after performing all the opertaions.
    column : string
        name of the predictor column
    """
    cols = [col for col in data.columns if "next" not in col.lower() and col.lower().endswith("gr")]
    cols.append(column)
    data = data[cols]
    return (data,column)

In [6]:
def best_parameters(X,Y):
    custom_scorer = make_scorer(f1_score, greater_is_better=True,pos_label = 1)
    params = {'n_neighbors':np.arange(1,105,5),'weights':['uniform', 'distance'],'metric':['euclidean','manhattan','minkowski'],"algorithm":["ball_tree","kd_tree","brute"]}
    model = GridSearchCV(KNeighborsClassifier(), params,scoring=custom_scorer)
    model.fit(X,Y)
    k = model.best_params_['n_neighbors']
    params = {'n_neighbors':np.arange(k-5, k+5), 'weights':['uniform', 'distance'], 'metric':['euclidean', 'manhattan','minkowski'],"algorithm":["ball_tree","kd_tree","brute"]}
    model = GridSearchCV(KNeighborsClassifier(), params,scoring=custom_scorer)
    model.fit(X,Y)
    metric,n_neighbors,weights,algorithm = model.best_params_['metric'],model.best_params_['n_neighbors'],model.best_params_['weights'],model.best_params_['algorithm'],
    return metric,n_neighbors,weights,algorithm

In [7]:
def error_metrics(y_true, y_pred):
    rmse = sqrt(metrics.mean_squared_error(y_true, y_pred))
    mae = metrics.mean_absolute_error(y_true, y_pred)
    mse = metrics.mean_squared_error(y_true, y_pred)
    return {"root_mean_squared_error":rmse,"mean_absolute_error":mae,"mean_squared_error":mse}

In [8]:
def create_confusion_matrix(y_pred,y_true):
    cm = confusion_matrix(y_true,y_pred)
    accuracy = metrics.accuracy_score(y_true,y_pred)
    precision = metrics.precision_score(y_true,y_pred)
    recall = metrics.recall_score(y_true,y_pred)
    f1_score = metrics.f1_score(y_true,y_pred)
    return {"accuracy":accuracy,"precision":precision,"recall":recall,"f1_score":f1_score,"confusion matrix":cm}

In [9]:
def create_knn_classification(df,column,metric,n_neighbors,weights,algorithm,rate):
    df["Target"] = df[column].apply(lambda x : 1 if x >= rate else 0)
    X = df.drop(columns=["Target",column])
    Y = df["Target"]
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=0)
    knn = KNeighborsClassifier(metric=metric,n_neighbors=n_neighbors,weights=weights)
    knn.fit(X_train, y_train)
    y_pred = knn.predict(X_test)
    result = {}
    confusion = create_confusion_matrix(y_pred,y_test)
    error = error_metrics(y_test, y_pred)
    result.update(error)
    result.update(confusion)
    return result    

In [10]:
def k_nearest_neighbours_classification(df,column="Next Day Close Price GR"):
    solution = list()
    rate_of_growth = [0.01,0.02,0.03,0.04,0.05]
    for t in rate_of_growth:
        df["Target"] = df[column].apply(lambda x : 1 if x >= t else 0)
        X = df.drop(columns=["Target",column])
        Y = df["Target"]
        X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=0)
        metric,n_neighbors,weights,algorithm = best_parameters(X_train,y_train)
        result = create_knn_classification(df,column,metric,n_neighbors,weights,algorithm,t)
        result.update({"rate_of_growth":t})
        result.update({"metric":metric,"n_neighbors":n_neighbors,"weights":weights,"algorithm":algorithm})
        solution.append(result)
    return solution

In [11]:
total = pd.read_csv("/kaggle/input/stockdata/all.csv")
stockpath = '/kaggle/input/stockdata/Data/Stock'

In [12]:
%%time
mydf = pd.DataFrame()
for _,row in total.iterrows():
    try:
        security_id = str(row['security id'])
        print(security_id)
        df = pd.read_csv(os.path.join(stockpath,"gr"+security_id+".csv"))
        df = pre_process_data(df,60)
        column = "Next Day Close Price GR"
        (df,column) = dependent_column(df,column)
        knn_result = k_nearest_neighbours_classification(df,column)
        knn_df = pd.DataFrame(knn_result)
        knn_df.to_csv('knn_'+str(security_id)+".csv",index=None)
    except Exception as e:
        traceback.print_exc()

500112
500325
532540
500209
532174
507685
530965
500182
532210
500180
500680
506395
500770
500085
501425
532899
537291
500790
500825
533155
533287
533260
539921
542602
543217
543261
532538


Traceback (most recent call last):
  File "<timed exec>", line 6, in <module>
  File "/opt/conda/lib/python3.7/site-packages/pandas/io/parsers.py", line 605, in read_csv
    return _read(filepath_or_buffer, kwds)
  File "/opt/conda/lib/python3.7/site-packages/pandas/io/parsers.py", line 457, in _read
    parser = TextFileReader(filepath_or_buffer, **kwds)
  File "/opt/conda/lib/python3.7/site-packages/pandas/io/parsers.py", line 814, in __init__
    self._engine = self._make_engine(self.engine)
  File "/opt/conda/lib/python3.7/site-packages/pandas/io/parsers.py", line 1045, in _make_engine
    return mapping[engine](self.f, **self.options)  # type: ignore[call-arg]
  File "/opt/conda/lib/python3.7/site-packages/pandas/io/parsers.py", line 1862, in __init__
    self._open_handles(src, kwds)
  File "/opt/conda/lib/python3.7/site-packages/pandas/io/parsers.py", line 1363, in _open_handles
    storage_options=kwds.get("storage_options", None),
  File "/opt/conda/lib/python3.7/site-packages

500387
500425
532689


Traceback (most recent call last):
  File "<timed exec>", line 6, in <module>
  File "/opt/conda/lib/python3.7/site-packages/pandas/io/parsers.py", line 605, in read_csv
    return _read(filepath_or_buffer, kwds)
  File "/opt/conda/lib/python3.7/site-packages/pandas/io/parsers.py", line 457, in _read
    parser = TextFileReader(filepath_or_buffer, **kwds)
  File "/opt/conda/lib/python3.7/site-packages/pandas/io/parsers.py", line 814, in __init__
    self._engine = self._make_engine(self.engine)
  File "/opt/conda/lib/python3.7/site-packages/pandas/io/parsers.py", line 1045, in _make_engine
    return mapping[engine](self.f, **self.options)  # type: ignore[call-arg]
  File "/opt/conda/lib/python3.7/site-packages/pandas/io/parsers.py", line 1862, in __init__
    self._open_handles(src, kwds)
  File "/opt/conda/lib/python3.7/site-packages/pandas/io/parsers.py", line 1363, in _open_handles
    storage_options=kwds.get("storage_options", None),
  File "/opt/conda/lib/python3.7/site-packages

532706
532163
524715
532488
500124
CPU times: user 3h 20min, sys: 16.7 s, total: 3h 20min 16s
Wall time: 3h 20min 20s
